Imports packages and connection to internal Actuary Team database

In [1]:
import pandas as pd
import pyodbc
connection_string = ("DRIVER={SQL Server};"
"SERVER=USDF11V0954;"
"DATABASE=Actuarial_AH;"
"Trust_Connection=yes;")

conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

Runs SQL query that creates a cohort of individuals that had a script for Wegovy, Saxenda, or Zepbound between service dates 2022 and 2025. Using that cohort, returns medical claims dataset that were paid between 2021 and 2025 and were actively enrolled since 2021.

In [70]:
med = """
WITH WEIGHTLOSSCOHORT1 AS (
    SELECT DISTINCT MEMBER_ID,
           YEAR(SERVICE_DATE) AS PRESCRIPTION_YEAR
    FROM Actuarial_AH.DBO.SN_Rx
    WHERE DRUG_NAME_PREFERRED IN ('Wegovy', 'Saxenda', 'Zepbound')
    AND SERVICE_DATE >= '2022-01-01'
    AND SERVICE_DATE < '2025-01-01'
)
SELECT M.MEDICAL_CLAIM_ID, 
       M.MEMBER_ID, 
       M.PAID_DATE, 
	   YEAR(M.PAID_DATE) AS PAID_YEAR,
       M.SERVICE_DATE,
YEAR(M.SERVICE_DATE) AS SERVICE_YEAR,
       M.MEDICAL_PAID_AMOUNT, 
       M.IS_TELEMEDICINE, 
       M.IS_ER_AVOIDABLE, 
M.DIAGNOSIS_DESC_ICD10_1 AS PRIMARY_DX,
       M.PROCEDURE_DESC, 
       M.ARTTOS_V2_L1, 
       M.ARTTOS_V2_L3, 
M.DX_IS_CHRONIC,
       M.ICD10_CHAPTER, 
       M.ICD10_CATEGORY, 
       M.ICD10_SECTION, 
       M.MEG_EPISODE_DESCRIPTION, 
       M.OP_SURG_INC,
       M.IS_PCP_VISIT, 
       M.INCLUDED_SPECIALIST, 
       M.ER_VISIT_FLAG, 
       M.IS_URGENT_CARE_VISIT, 
       M.IS_PREVENTIVE_VISIT, 
       M.IP_ADMIT_INC
FROM [Actuarial_AH].[dbo].[SN_Medical] M
JOIN 
Actuarial_AH.dbo.SN_Member MB ON M.MEMBER_ID = MB.MEMBER_ID
WHERE M.PAID_DATE BETWEEN '2021-01-01' AND '2024-12-31'
AND MB.MONTH_KEY = '2024-12-01' 
AND MB.MEDICAL_ENROLLMENT_STATUS = 3
AND MB.MEDICAL_CONT_ENRLMNT_START <= '2021-01-01'
AND MB.MEMBER_ID IN (
    SELECT MEMBER_ID
    FROM WEIGHTLOSSCOHORT1
    GROUP BY MEMBER_ID
	)
"""
med = pd.read_sql(med, conn)

C:\Users\U1229773\AppData\Local\Temp\ipykernel_30428\2598388549.py:49: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  med = pd.read_sql(med, conn)


In [3]:
#create mock claims dataset to replace company data
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# Set seed for reproducibility
np.random.seed(42)

# Parameters
num_records = 100000
num_members = 10000

# Generate member IDs (weight loss cohort)
member_ids = np.random.choice(range(10000, 20000), size=num_members, replace=False)
selected_member_ids = np.random.choice(member_ids, size=num_records, replace=True)

# Generate random dates between 2021 and 2024
def random_date(start, end):
    return start + timedelta(days=random.randint(0, (end - start).days))

start_date = datetime(2021, 1, 1)
end_date = datetime(2024, 12, 31)
paid_dates = [random_date(start_date, end_date) for _ in range(num_records)]
service_dates = [random_date(start_date, end_date) for _ in range(num_records)]

# Claim IDs
medical_claim_ids = np.arange(1, num_records + 1)

# Amounts and binary flags
medical_paid_amounts = np.round(np.random.exponential(scale=200, size=num_records), 2)
bool_choices = [0, 1]

# Text-based columns (mocked)
diagnosis_desc = np.random.choice(['Obesity', 'Type 2 Diabetes', 'Hypertension', 'Hyperlipidemia'], size=num_records)
procedure_desc = np.random.choice(['Office Visit', 'Lab Test', 'Imaging', 'Bariatric Surgery'], size=num_records)
arttos_l1 = np.random.choice(['Evaluation & Management', 'Surgery', 'Radiology'], size=num_records)
arttos_l3 = np.random.choice(['Office Visit', 'Major Surgery', 'Outpatient Imaging'], size=num_records)
icd10_chapter = np.random.choice(['Endocrine', 'Circulatory', 'Digestive'], size=num_records)
icd10_category = np.random.choice(['E11', 'I10', 'K21'], size=num_records)
icd10_section = np.random.choice(['E11.9', 'I10.0', 'K21.9'], size=num_records)
meg_description = np.random.choice(['Diabetes Management', 'Obesity Treatment', 'Routine Checkup'], size=num_records)

# Create DataFrame
med = pd.DataFrame({
    'MEDICAL_CLAIM_ID': medical_claim_ids,
    'MEMBER_ID': selected_member_ids,
    'PAID_DATE': paid_dates,
    'PAID_YEAR': [d.year for d in paid_dates],
    'SERVICE_DATE': service_dates,
    'SERVICE_YEAR': [d.year for d in service_dates],
    'MEDICAL_PAID_AMOUNT': medical_paid_amounts,
    'IS_TELEMEDICINE': np.random.choice(bool_choices, size=num_records),
    'IS_ER_AVOIDABLE': np.random.choice(bool_choices, size=num_records),
    'PRIMARY_DX': diagnosis_desc,
    'PROCEDURE_DESC': procedure_desc,
    'ARTTOS_V2_L1': arttos_l1,
    'ARTTOS_V2_L3': arttos_l3,
    'DX_IS_CHRONIC': np.random.choice(bool_choices, size=num_records),
    'ICD10_CHAPTER': icd10_chapter,
    'ICD10_CATEGORY': icd10_category,
    'ICD10_SECTION': icd10_section,
    'MEG_EPISODE_DESCRIPTION': meg_description,
    'OP_SURG_INC': np.random.choice(bool_choices, size=num_records),
    'IS_PCP_VISIT': np.random.choice(bool_choices, size=num_records),
    'INCLUDED_SPECIALIST': np.random.choice(bool_choices, size=num_records),
    'ER_VISIT_FLAG': np.random.choice(bool_choices, size=num_records),
    'IS_URGENT_CARE_VISIT': np.random.choice(bool_choices, size=num_records),
    'IS_PREVENTIVE_VISIT': np.random.choice(bool_choices, size=num_records),
    'IP_ADMIT_INC': np.random.choice(bool_choices, size=num_records)
})

med.head()


,MEDICAL_CLAIM_ID,MEMBER_ID,PAID_DATE,PAID_YEAR,SERVICE_DATE,SERVICE_YEAR,MEDICAL_PAID_AMOUNT,IS_TELEMEDICINE,IS_ER_AVOIDABLE,PRIMARY_DX,...,ICD10_CATEGORY,ICD10_SECTION,MEG_EPISODE_DESCRIPTION,OP_SURG_INC,IS_PCP_VISIT,INCLUDED_SPECIALIST,ER_VISIT_FLAG,IS_URGENT_CARE_VISIT,IS_PREVENTIVE_VISIT,IP_ADMIT_INC
0,1,18180,2021-07-20,2021,2024-01-01,2024,116.36,1,0,Hyperlipidemia,...,I10,E11.9,Routine Checkup,1,1,0,0,1,1,1
1,2,13277,2021-08-25,2021,2024-06-25,2024,96.93,0,1,Hyperlipidemia,...,K21,E11.9,Diabetes Management,1,1,0,0,0,1,1
2,3,12610,2021-05-29,2021,2022-10-04,2022,92.08,0,0,Hypertension,...,I10,E11.9,Obesity Treatment,0,1,0,0,1,0,1
3,4,18708,2022-04-21,2022,2024-12-14,2024,256.49,1,0,Hypertension,...,K21,I10.0,Diabetes Management,1,1,1,0,0,0,0
4,5,16682,2023-06-23,2023,2024-09-16,2024,141.06,1,0,Hyperlipidemia,...,I10,I10.0,Diabetes Management,0,0,0,0,1,0,0


Runs SQL query that creates a cohort of individuals that had a script for Wegovy, Saxenda, or Zepbound between service dates 2022 and 2025. Using that cohort, returns pharmacy claims dataset that were paid between 2021 and 2025 and were actively enrolled since 2021.

In [71]:
rx = """
WITH WEIGHTLOSSCOHORT1 AS (
    SELECT DISTINCT MEMBER_ID,
           YEAR(SERVICE_DATE) AS PRESCRIPTION_YEAR
    FROM Actuarial_AH.DBO.SN_Rx
    WHERE DRUG_NAME_PREFERRED IN ('Wegovy', 'Saxenda', 'Zepbound')
    AND SERVICE_DATE BETWEEN '2022-01-01' AND '2024-12-31'
)
SELECT R.RX_CLAIM_ID, 
       R.MEMBER_ID, 
       R.PAID_DATE,
       R.DAYS,
       R.SERVICE_DATE, 
       R.DRUG_NAME_GENERIC,
	   R.DRUG_NAME_PREFERRED,
	   R.RX_PAID_AMOUNT,
	   R.RX_SCRIPT_COUNT,
	   R.ART_DRUGGRPS_L1,
	   R.ART_DRUGGRPS_L2,
	   YEAR(R.PAID_DATE) AS PAID_YEAR,
YEAR(R.SERVICE_DATE) AS SERVICE_YEAR
FROM [Actuarial_AH].[dbo].[SN_Rx] R
JOIN 
Actuarial_AH.dbo.SN_Member M ON R.MEMBER_ID = M.MEMBER_ID
WHERE R.PAID_DATE BETWEEN '2021-01-01' AND '2024-12-31'
AND M.MONTH_KEY = '2024-12-01' 
AND M.MEDICAL_ENROLLMENT_STATUS = 3
AND M.MEDICAL_CONT_ENRLMNT_START <= '2021-01-01'
AND M.MEMBER_ID IN (
    SELECT MEMBER_ID
    FROM WEIGHTLOSSCOHORT1
    GROUP BY MEMBER_ID
	)
"""
rx = pd.read_sql(rx, conn)

C:\Users\U1229773\AppData\Local\Temp\ipykernel_30428\3517386714.py:36: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  rx = pd.read_sql(rx, conn)


In [2]:
# Set seed for reproducibility
np.random.seed(42)

# Use the same MEMBER_IDs as the medical dataset (assumed already created)
# Simulate cohort of members
rx_member_ids = np.random.choice(range(10000, 20000), size=10000, replace=False)
num_rx_records = 100000
selected_rx_member_ids = np.random.choice(rx_member_ids, size=num_rx_records, replace=True)

# Date range
start_date = datetime(2021, 1, 1)
end_date = datetime(2024, 12, 31)

def random_date(start, end):
    return start + timedelta(days=random.randint(0, (end - start).days))

rx_paid_dates = [random_date(start_date, end_date) for _ in range(num_rx_records)]
rx_service_dates = [random_date(datetime(2022, 1, 1), datetime(2024, 12, 31)) for _ in range(num_rx_records)]

# Random drug assignment (cohort drugs + others)
cohort_drugs = [
    ('semaglutide', 'Wegovy'),
    ('liraglutide', 'Saxenda'),
    ('tirzepatide', 'Zepbound')
]

other_drugs = [
    ('metformin', 'Glucophage'),
    ('atorvastatin', 'Lipitor'),
    ('lisinopril', 'Prinivil'),
    ('amlodipine', 'Norvasc'),
    ('hydrochlorothiazide', 'Microzide'),
    ('levothyroxine', 'Synthroid'),
    ('albuterol', 'Ventolin'),
]

# Mix 60% cohort drugs, 40% other
cohort_size = int(num_rx_records * 0.6)
other_size = num_rx_records - cohort_size
drug_mix = random.choices(cohort_drugs, k=cohort_size) + random.choices(other_drugs, k=other_size)
random.shuffle(drug_mix)  # Shuffle to mix cohort and non-cohort drugs

generic_names, preferred_names = zip(*drug_mix)

# Other fields
rx_claim_ids = np.arange(1, num_rx_records + 1)
rx_days = np.random.choice([30, 60, 90], size=num_rx_records)
rx_paid_amounts = np.round(np.random.exponential(scale=150, size=num_rx_records), 2)
rx_script_counts = np.random.choice([1, 2, 3], size=num_rx_records)
art_druggrps_l1 = np.random.choice(['Antidiabetics', 'Obesity Agents', 'Antihypertensives', 'Respiratory Agents'], size=num_rx_records)
art_druggrps_l2 = np.random.choice(['GLP-1 Agonists', 'ACE Inhibitors', 'Beta Blockers', 'Inhalers'], size=num_rx_records)

# Build DataFrame
rx = pd.DataFrame({
    'RX_CLAIM_ID': rx_claim_ids,
    'MEMBER_ID': selected_rx_member_ids,
    'PAID_DATE': rx_paid_dates,
    'DAYS': rx_days,
    'SERVICE_DATE': rx_service_dates,
    'DRUG_NAME_GENERIC': generic_names,
    'DRUG_NAME_PREFERRED': preferred_names,
    'RX_PAID_AMOUNT': rx_paid_amounts,
    'RX_SCRIPT_COUNT': rx_script_counts,
    'ART_DRUGGRPS_L1': art_druggrps_l1,
    'ART_DRUGGRPS_L2': art_druggrps_l2,
    'PAID_YEAR': [d.year for d in rx_paid_dates],
    'SERVICE_YEAR': [d.year for d in rx_service_dates]
})

# Preview
rx.head()


,RX_CLAIM_ID,MEMBER_ID,PAID_DATE,DAYS,SERVICE_DATE,DRUG_NAME_GENERIC,DRUG_NAME_PREFERRED,RX_PAID_AMOUNT,RX_SCRIPT_COUNT,ART_DRUGGRPS_L1,ART_DRUGGRPS_L2,PAID_YEAR,SERVICE_YEAR
0,1,18180,2023-02-10,60,2023-01-25,hydrochlorothiazide,Microzide,31.73,3,Antidiabetics,ACE Inhibitors,2023,2023
1,2,13277,2021-08-29,30,2024-07-10,semaglutide,Wegovy,11.59,1,Obesity Agents,ACE Inhibitors,2021,2024
2,3,12610,2024-12-02,60,2023-12-13,semaglutide,Wegovy,46.64,2,Obesity Agents,ACE Inhibitors,2024,2023
3,4,18708,2024-11-15,60,2022-11-03,hydrochlorothiazide,Microzide,43.10,3,Obesity Agents,Beta Blockers,2024,2022
4,5,16682,2024-06-27,30,2024-12-28,amlodipine,Norvasc,131.72,2,Antihypertensives,ACE Inhibitors,2024,2024


Runs SQL query that creates a cohort of individuals that had a script for Wegovy, Saxenda, or Zepbound between service dates 2022 and 2025. Using that cohort, returns unique members dataset that were actively enrolled since 2021.

In [72]:
member = """
WITH WEIGHTLOSSCOHORT1 AS (
    SELECT DISTINCT MEMBER_ID,
           YEAR(SERVICE_DATE) AS PRESCRIPTION_YEAR
    FROM Actuarial_AH.DBO.SN_Rx
    WHERE DRUG_NAME_PREFERRED IN ('Wegovy', 'Saxenda', 'Zepbound')
    AND SERVICE_DATE >= '2022-01-01'
    AND SERVICE_DATE < '2025-01-01'
)
SELECT M.MEMBER_ID,
       M.MEMBER_STATUS,
       M.HOME_STATE,
	   CASE 
    WHEN M.MEMBER_RELATIONSHIP = 0 THEN 'Subscriber'
    WHEN M.MEMBER_RELATIONSHIP = 1 THEN 'Spouse'
    WHEN M.MEMBER_RELATIONSHIP = 2 THEN 'Dependent'
    WHEN M.MEMBER_RELATIONSHIP = 3 THEN 'Domestic Partner'
    WHEN M.MEMBER_RELATIONSHIP = 4 THEN 'Unknown'
    ELSE 'Unknown' -- Optional: Handle unexpected values
END AS MEMBER_RELATIONSHIP,
CASE 
        WHEN M.GENDER = 0 THEN 'Female'
        WHEN M.GENDER = 1 THEN 'Male'
        WHEN M.GENDER = 2 THEN 'Unknown'
        ELSE 'Unknown' 
    END AS GENDER,
    DATEDIFF(YEAR, CONVERT(DATE, M.BIRTH_YEAR_MO + '-01'), GETDATE()) - 
    CASE 
        WHEN MONTH(CONVERT(DATE, M.BIRTH_YEAR_MO + '-01')) > MONTH(GETDATE()) 
             OR (MONTH(CONVERT(DATE, M.BIRTH_YEAR_MO + '-01')) = MONTH(GETDATE()) 
                 AND DAY(CONVERT(DATE, M.BIRTH_YEAR_MO + '-01')) > DAY(GETDATE())) 
        THEN 1 
        ELSE 0 
    END AS Age,
	CASE 
    WHEN M.BIRTH_YEAR >= 1946 AND M.BIRTH_YEAR < 1965 THEN 'Baby Boomers'
	WHEN M.BIRTH_YEAR >= 1965 AND M.BIRTH_YEAR < 1981 THEN 'Generation X'
	WHEN M.BIRTH_YEAR >= 1981 AND M.BIRTH_YEAR < 1997 THEN 'Millenials'
	WHEN M.BIRTH_YEAR >= 1997 AND M.BIRTH_YEAR < 2012 THEN 'Generation Z'
    ELSE 'Unknown' -- Optional: Handle unexpected values
END AS GENERATIONS
FROM [Actuarial_AH].[dbo].[SN_MEMBER] M
WHERE M.MEMBER_ID IN (
    SELECT MEMBER_ID
    FROM WEIGHTLOSSCOHORT1
    GROUP BY MEMBER_ID
	)
AND M.MONTH_KEY = '2024-12-01' 
AND M.MEDICAL_ENROLLMENT_STATUS = 3
AND M.MEDICAL_CONT_ENRLMNT_START <= '2021-01-01'
"""
member = pd.read_sql(member, conn)

C:\Users\U1229773\AppData\Local\Temp\ipykernel_30428\1131752232.py:55: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  member = pd.read_sql(member, conn)


In [5]:
# Set seed for reproducibility
np.random.seed(42)

# Assume these MEMBER_IDs come from your medical/pharmacy datasets
member_ids = np.random.choice(range(10000, 20000), size=10000, replace=False)
num_members = len(member_ids)

# Mock MEMBER_STATUS (1 = active, 2 = COBRA, 3 = retired)
member_status = np.random.choice([1, 2, 3], size=num_members)

# HOME_STATE (US states)
states = ['UT', 'CA', 'TX', 'NY', 'FL', 'WA', 'IL', 'CO', 'PA', 'NC']
home_states = np.random.choice(states, size=num_members)

# MEMBER_RELATIONSHIP codes
relationship_codes = np.random.choice([0, 1, 2, 3, 4], size=num_members)
relationship_map = {
    0: 'Subscriber',
    1: 'Spouse',
    2: 'Dependent',
    3: 'Domestic Partner',
    4: 'Unknown'
}
relationship_labels = [relationship_map[r] for r in relationship_codes]

# GENDER codes
genders = np.random.choice([0, 1, 2], size=num_members)
gender_map = {
    0: 'Female',
    1: 'Male',
    2: 'Unknown'
}
gender_labels = [gender_map[g] for g in genders]

# BIRTH_YEAR_MO generation
birth_years = np.random.choice(range(1946, 2012), size=num_members)
birth_months = np.random.choice(range(1, 13), size=num_members)
birth_year_mo = [f"{y}-{m:02d}" for y, m in zip(birth_years, birth_months)]

# Age calculation as of June 28, 2025
ref_date = datetime(2025, 6, 28)
ages = []
generations = []

for year, month in zip(birth_years, birth_months):
    dob = datetime(year, month, 1)
    age = ref_date.year - dob.year - ((ref_date.month, ref_date.day) < (dob.month, dob.day))
    ages.append(age)

    if 1946 <= year < 1965:
        generations.append('Baby Boomers')
    elif 1965 <= year < 1981:
        generations.append('Generation X')
    elif 1981 <= year < 1997:
        generations.append('Millenials')
    elif 1997 <= year < 2012:
        generations.append('Generation Z')
    else:
        generations.append('Unknown')

# Create DataFrame
member = pd.DataFrame({
    'MEMBER_ID': member_ids,
    'MEMBER_STATUS': member_status,
    'HOME_STATE': home_states,
    'MEMBER_RELATIONSHIP': relationship_labels,
    'GENDER': gender_labels,
    'Age': ages,
    'GENERATIONS': generations
})

# Preview
member.head()

,MEMBER_ID,MEMBER_STATUS,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,Age,GENERATIONS
0,16252,2,CA,Subscriber,Male,56,Generation X
1,14684,3,NC,Domestic Partner,Male,73,Baby Boomers
2,11731,2,WA,Spouse,Unknown,71,Baby Boomers
3,14742,3,FL,Subscriber,Female,55,Generation X
4,14521,1,FL,Subscriber,Unknown,39,Millenials


Reviewing medical, pharmacy and member tables

In [6]:
med

,MEDICAL_CLAIM_ID,MEMBER_ID,PAID_DATE,PAID_YEAR,SERVICE_DATE,SERVICE_YEAR,MEDICAL_PAID_AMOUNT,IS_TELEMEDICINE,IS_ER_AVOIDABLE,PRIMARY_DX,...,ICD10_CATEGORY,ICD10_SECTION,MEG_EPISODE_DESCRIPTION,OP_SURG_INC,IS_PCP_VISIT,INCLUDED_SPECIALIST,ER_VISIT_FLAG,IS_URGENT_CARE_VISIT,IS_PREVENTIVE_VISIT,IP_ADMIT_INC
0,1,18180,2021-07-20,2021,2024-01-01,2024,116.36,1,0,Hyperlipidemia,...,I10,E11.9,Routine Checkup,1,1,0,0,1,1,1
1,2,13277,2021-08-25,2021,2024-06-25,2024,96.93,0,1,Hyperlipidemia,...,K21,E11.9,Diabetes Management,1,1,0,0,0,1,1
2,3,12610,2021-05-29,2021,2022-10-04,2022,92.08,0,0,Hypertension,...,I10,E11.9,Obesity Treatment,0,1,0,0,1,0,1
3,4,18708,2022-04-21,2022,2024-12-14,2024,256.49,1,0,Hypertension,...,K21,I10.0,Diabetes Management,1,1,1,0,0,0,0
4,5,16682,2023-06-23,2023,2024-09-16,2024,141.06,1,0,Hyperlipidemia,...,I10,I10.0,Diabetes Management,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99996,12607,2022-05-18,2022,2023-07-02,2023,44.26,1,1,Hypertension,...,K21,I10.0,Obesity Treatment,0,1,1,1,1,0,1
99996,99997,14528,2021-10-26,2021,2023-07-07,2023,69.52,1,1,Type 2 Diabetes,...,E11,E11.9,Obesity Treatment,0,0,1,0,1,0,0
99997,99998,14138,2023-12-10,2023,2022-08-30,2022,426.73,1,1,Type 2 Diabetes,...,I10,K21.9,Obesity Treatment,0,0,1,1,1,1,1
99998,99999,16528,2021-07-30,2021,2022-05-19,2022,83.67,0,1,Obesity,...,I10,E11.9,Routine Checkup,1,1,1,0,0,0,0


In [7]:
rx

,RX_CLAIM_ID,MEMBER_ID,PAID_DATE,DAYS,SERVICE_DATE,DRUG_NAME_GENERIC,DRUG_NAME_PREFERRED,RX_PAID_AMOUNT,RX_SCRIPT_COUNT,ART_DRUGGRPS_L1,ART_DRUGGRPS_L2,PAID_YEAR,SERVICE_YEAR
0,1,18180,2023-02-10,60,2023-01-25,hydrochlorothiazide,Microzide,31.73,3,Antidiabetics,ACE Inhibitors,2023,2023
1,2,13277,2021-08-29,30,2024-07-10,semaglutide,Wegovy,11.59,1,Obesity Agents,ACE Inhibitors,2021,2024
2,3,12610,2024-12-02,60,2023-12-13,semaglutide,Wegovy,46.64,2,Obesity Agents,ACE Inhibitors,2024,2023
3,4,18708,2024-11-15,60,2022-11-03,hydrochlorothiazide,Microzide,43.10,3,Obesity Agents,Beta Blockers,2024,2022
4,5,16682,2024-06-27,30,2024-12-28,amlodipine,Norvasc,131.72,2,Antihypertensives,ACE Inhibitors,2024,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99996,12607,2023-10-23,60,2023-05-21,liraglutide,Saxenda,456.91,2,Antidiabetics,Beta Blockers,2023,2023
99996,99997,14528,2021-04-11,30,2024-07-06,liraglutide,Saxenda,410.99,1,Antidiabetics,GLP-1 Agonists,2021,2024
99997,99998,14138,2022-04-25,30,2022-05-31,lisinopril,Prinivil,236.02,2,Obesity Agents,Beta Blockers,2022,2022
99998,99999,16528,2024-03-10,90,2024-05-26,albuterol,Ventolin,82.51,3,Respiratory Agents,Inhalers,2024,2024


In [8]:
member

,MEMBER_ID,MEMBER_STATUS,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,Age,GENERATIONS
0,16252,2,CA,Subscriber,Male,56,Generation X
1,14684,3,NC,Domestic Partner,Male,73,Baby Boomers
2,11731,2,WA,Spouse,Unknown,71,Baby Boomers
3,14742,3,FL,Subscriber,Female,55,Generation X
4,14521,1,FL,Subscriber,Unknown,39,Millenials
...,...,...,...,...,...,...,...
9995,15734,2,WA,Subscriber,Unknown,46,Generation X
9996,15191,3,CA,Dependent,Male,15,Generation Z
9997,15390,2,NY,Subscriber,Unknown,43,Millenials
9998,10860,1,UT,Dependent,Unknown,42,Millenials


Find members that had taken a weight-loss GLP-1 in 2020 and 2021 and remove from the tables. This study looks at individuals who initiated in 2022.

In [9]:
# Define the drugs to filter out
drugs_to_remove = ['Wegovy', 'Saxenda', 'Zepbound']

# Filter to find members with the specified drugs in 2021
members_to_remove = rx[((rx['SERVICE_YEAR'] == 2021) | (rx['SERVICE_YEAR'] == 2020)) & (rx['DRUG_NAME_PREFERRED'].isin(drugs_to_remove))]['MEMBER_ID'].unique()

# Remove those members from the original DataFrame
rx_members_glp_22_24 = rx[~rx['MEMBER_ID'].isin(members_to_remove)]
med_members_glp_22_24 = med[~med['MEMBER_ID'].isin(members_to_remove)]
members_glp_22_24 = member[~member['MEMBER_ID'].isin(members_to_remove)]

Create a view that will set up the Percentage of Days Covered (PDC) metric. Locate GLP-1s in the rx dataset and determine days filled, start and end days per script.

In [10]:
# Define NDCs or names of GLP-1 drugs
glp1_drugs = ['Wegovy', 'Saxenda', 'Zepbound'] 

# Step 1: Filter for GLP-1 claims only
glp1_drugs_rx = rx_members_glp_22_24[rx_members_glp_22_24['DRUG_NAME_PREFERRED'].isin(glp1_drugs)].copy()

# Step 2: Convert fill_date to datetime
glp1_drugs_rx['SERVICE_DATE'] = pd.to_datetime(glp1_drugs_rx['SERVICE_DATE'])

# Step 3: Create a column for days covered per fill
glp1_drugs_rx['END_DATE'] = glp1_drugs_rx['SERVICE_DATE'] + pd.to_timedelta(glp1_drugs_rx['DAYS'] - 1, unit='D')
glp1_drugs_rx

,RX_CLAIM_ID,MEMBER_ID,PAID_DATE,DAYS,SERVICE_DATE,DRUG_NAME_GENERIC,DRUG_NAME_PREFERRED,RX_PAID_AMOUNT,RX_SCRIPT_COUNT,ART_DRUGGRPS_L1,ART_DRUGGRPS_L2,PAID_YEAR,SERVICE_YEAR,END_DATE
1,2,13277,2021-08-29,30,2024-07-10,semaglutide,Wegovy,11.59,1,Obesity Agents,ACE Inhibitors,2021,2024,2024-08-08
2,3,12610,2024-12-02,60,2023-12-13,semaglutide,Wegovy,46.64,2,Obesity Agents,ACE Inhibitors,2024,2023,2024-02-10
5,6,12137,2024-04-24,60,2023-06-22,liraglutide,Saxenda,55.89,1,Antidiabetics,ACE Inhibitors,2024,2023,2023-08-20
10,11,15089,2022-12-19,90,2024-03-20,semaglutide,Wegovy,59.59,3,Antidiabetics,Beta Blockers,2022,2024,2024-06-17
11,12,11357,2022-03-05,90,2022-04-18,semaglutide,Wegovy,161.03,3,Antidiabetics,Inhalers,2022,2022,2022-07-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99991,99992,19266,2024-01-22,30,2022-03-16,tirzepatide,Zepbound,1.32,2,Obesity Agents,ACE Inhibitors,2024,2022,2022-04-14
99992,99993,13474,2022-04-26,60,2023-05-02,tirzepatide,Zepbound,187.13,3,Respiratory Agents,ACE Inhibitors,2022,2023,2023-06-30
99993,99994,19951,2024-06-16,60,2022-05-31,liraglutide,Saxenda,68.19,3,Respiratory Agents,GLP-1 Agonists,2024,2022,2022-07-29
99995,99996,12607,2023-10-23,60,2023-05-21,liraglutide,Saxenda,456.91,2,Antidiabetics,Beta Blockers,2023,2023,2023-07-19


Aggregate sum of days filled by member ID and earliest and latest service/fill dates. Determine 3 respective years from earliest service date to create a 365-day timelines to calculate PDC.

In [11]:
member_pdc = glp1_drugs_rx.groupby('MEMBER_ID').agg(
    min_service_date=('SERVICE_DATE', 'min'),
    max_end_date=('END_DATE', 'max'),
    covered_days = ('DAYS', 'sum')
).reset_index()

# Add 12 months for each year
for year in range(1, 4):
    member_pdc[f'min_service_date_year_{year}'] = member_pdc['min_service_date'] + pd.DateOffset(months=12 * year)

member_pdc

,MEMBER_ID,min_service_date,max_end_date,covered_days,min_service_date_year_1,min_service_date_year_2,min_service_date_year_3
0,10000,2023-04-09,2024-12-21,270,2024-04-09,2025-04-09,2026-04-09
1,10001,2022-01-14,2025-01-18,540,2023-01-14,2024-01-14,2025-01-14
2,10002,2022-02-07,2024-12-30,660,2023-02-07,2024-02-07,2025-02-07
3,10003,2022-12-17,2025-02-28,300,2023-12-17,2024-12-17,2025-12-17
4,10004,2022-02-11,2024-09-11,600,2023-02-11,2024-02-11,2025-02-11
...,...,...,...,...,...,...,...
9972,19995,2022-03-28,2024-12-30,210,2023-03-28,2024-03-28,2025-03-28
9973,19996,2023-06-02,2024-09-29,60,2024-06-02,2025-06-02,2026-06-02
9974,19997,2022-01-03,2025-03-29,780,2023-01-03,2024-01-03,2025-01-03
9975,19998,2022-05-02,2023-05-11,150,2023-05-02,2024-05-02,2025-05-02


Only keep members that had earliest script dates in 2022.

In [12]:
members_2022 = member_pdc[member_pdc['min_service_date'] < '2023']
members_2022

,MEMBER_ID,min_service_date,max_end_date,covered_days,min_service_date_year_1,min_service_date_year_2,min_service_date_year_3
1,10001,2022-01-14,2025-01-18,540,2023-01-14,2024-01-14,2025-01-14
2,10002,2022-02-07,2024-12-30,660,2023-02-07,2024-02-07,2025-02-07
3,10003,2022-12-17,2025-02-28,300,2023-12-17,2024-12-17,2025-12-17
4,10004,2022-02-11,2024-09-11,600,2023-02-11,2024-02-11,2025-02-11
5,10005,2022-01-07,2024-10-04,810,2023-01-07,2024-01-07,2025-01-07
...,...,...,...,...,...,...,...
9971,19994,2022-07-21,2024-05-03,450,2023-07-21,2024-07-21,2025-07-21
9972,19995,2022-03-28,2024-12-30,210,2023-03-28,2024-03-28,2025-03-28
9974,19997,2022-01-03,2025-03-29,780,2023-01-03,2024-01-03,2025-01-03
9975,19998,2022-05-02,2023-05-11,150,2023-05-02,2024-05-02,2025-05-02


In [13]:
# Initialize a dictionary to store results
results = []

# Loop through each member and calculate covered days
for index, member in members_2022.iterrows():
    member_id = member['MEMBER_ID']
    
    # Filter rx_df for claims before year 1 and year 2
    covered_days_year_1 = glp1_drugs_rx[(glp1_drugs_rx['MEMBER_ID'] == member_id) & 
                                 (glp1_drugs_rx['SERVICE_DATE'] < member['min_service_date_year_1'])]['DAYS'].sum()
    
    covered_days_year_2 = glp1_drugs_rx[(glp1_drugs_rx['MEMBER_ID'] == member_id) & 
                                 ((glp1_drugs_rx['SERVICE_DATE'] > member['min_service_date_year_1']) & (glp1_drugs_rx['SERVICE_DATE'] < member['min_service_date_year_2']))]['DAYS'].sum()
    
    # Append results
    results.append({
        'MEMBER_ID': member_id,
        'covered_days_year_1': covered_days_year_1,
        'covered_days_year_2': covered_days_year_2
    })

In [14]:
results_df = pd.DataFrame(results)
results_df

,MEMBER_ID,covered_days_year_1,covered_days_year_2
0,10001,150,180
1,10002,180,330
2,10003,150,90
3,10004,300,270
4,10005,150,450
...,...,...,...
8671,19994,300,150
8672,19995,90,60
8673,19997,150,180
8674,19998,150,0


In [15]:
members_2022_cvd = members_2022.merge(results_df, on='MEMBER_ID')
members_2022_cvd.drop(columns=['covered_days', 'min_service_date_year_3'], inplace=True)
members_2022_cvd['PDC1'] = members_2022_cvd['covered_days_year_1']/365
members_2022_cvd['PDC2'] = members_2022_cvd['covered_days_year_2']/365
members_2022_cvd

,MEMBER_ID,min_service_date,max_end_date,min_service_date_year_1,min_service_date_year_2,covered_days_year_1,covered_days_year_2,PDC1,PDC2
0,10001,2022-01-14,2025-01-18,2023-01-14,2024-01-14,150,180,0.410959,0.493151
1,10002,2022-02-07,2024-12-30,2023-02-07,2024-02-07,180,330,0.493151,0.904110
2,10003,2022-12-17,2025-02-28,2023-12-17,2024-12-17,150,90,0.410959,0.246575
3,10004,2022-02-11,2024-09-11,2023-02-11,2024-02-11,300,270,0.821918,0.739726
4,10005,2022-01-07,2024-10-04,2023-01-07,2024-01-07,150,450,0.410959,1.232877
...,...,...,...,...,...,...,...,...,...
8671,19994,2022-07-21,2024-05-03,2023-07-21,2024-07-21,300,150,0.821918,0.410959
8672,19995,2022-03-28,2024-12-30,2023-03-28,2024-03-28,90,60,0.246575,0.164384
8673,19997,2022-01-03,2025-03-29,2023-01-03,2024-01-03,150,180,0.410959,0.493151
8674,19998,2022-05-02,2023-05-11,2023-05-02,2024-05-02,150,0,0.410959,0.000000


In [16]:
members_2022_cvd.describe()

,MEMBER_ID,min_service_date,max_end_date,min_service_date_year_1,min_service_date_year_2,covered_days_year_1,covered_days_year_2,PDC1,PDC2
count,8676.000000,8676,8676,8676,8676,8676.000000,8676.000000,8676.000000,8676.000000
mean,14992.325380,2022-05-06 00:30:42.323651584,2024-08-30 00:54:26.390041344,2023-05-06 00:30:42.323651328,2024-05-05 16:59:45.062240768,179.253112,119.076763,0.491104,0.326238
min,10001.000000,2022-01-01 00:00:00,2022-02-04 00:00:00,2023-01-01 00:00:00,2024-01-01 00:00:00,30.000000,0.000000,0.082192,0.000000
25%,12471.750000,2022-02-14 00:00:00,2024-06-19 00:00:00,2023-02-14 00:00:00,2024-02-14 00:00:00,120.000000,60.000000,0.328767,0.164384
50%,14994.500000,2022-04-15 00:00:00,2024-10-23 00:00:00,2023-04-15 00:00:00,2024-04-15 00:00:00,180.000000,90.000000,0.493151,0.246575
75%,17506.250000,2022-07-10 00:00:00,2025-01-06 00:00:00,2023-07-10 00:00:00,2024-07-10 00:00:00,240.000000,180.000000,0.657534,0.493151
max,19999.000000,2022-12-31 00:00:00,2025-03-30 00:00:00,2023-12-31 00:00:00,2024-12-31 00:00:00,750.000000,630.000000,2.054795,1.726027
std,2892.197875,NaN,NaN,NaN,NaN,93.976489,90.309430,0.257470,0.247423


In [17]:
members_65 = members_2022_cvd[(members_2022_cvd['PDC1'] > 0.65) & (members_2022_cvd['PDC2'] > 0.65)]
members_65

,MEMBER_ID,min_service_date,max_end_date,min_service_date_year_1,min_service_date_year_2,covered_days_year_1,covered_days_year_2,PDC1,PDC2
3,10004,2022-02-11,2024-09-11,2023-02-11,2024-02-11,300,270,0.821918,0.739726
17,10019,2022-01-27,2025-01-29,2023-01-27,2024-01-27,300,270,0.821918,0.739726
41,10048,2022-01-10,2025-03-08,2023-01-10,2024-01-10,270,300,0.739726,0.821918
48,10057,2022-09-09,2025-02-26,2023-09-09,2024-09-09,270,300,0.739726,0.821918
67,10079,2022-01-17,2024-10-31,2023-01-17,2024-01-17,390,270,1.068493,0.739726
...,...,...,...,...,...,...,...,...,...
8534,19833,2022-03-03,2025-03-07,2023-03-03,2024-03-03,240,240,0.657534,0.657534
8562,19865,2022-04-29,2024-04-30,2023-04-29,2024-04-29,300,270,0.821918,0.739726
8580,19885,2022-01-18,2024-01-24,2023-01-18,2024-01-18,300,240,0.821918,0.657534
8651,19970,2022-01-12,2023-09-23,2023-01-12,2024-01-12,240,270,0.657534,0.739726


In [18]:
members_65.describe()

,MEMBER_ID,min_service_date,max_end_date,min_service_date_year_1,min_service_date_year_2,covered_days_year_1,covered_days_year_2,PDC1,PDC2
count,317.000000,317,317,317,317,317.000000,317.000000,317.000000,317.000000
mean,14788.668770,2022-05-02 01:49:01.324921088,2024-10-06 02:20:49.211356416,2023-05-02 01:49:01.324921088,2024-05-01 17:38:25.362776064,301.230284,284.384858,0.825288,0.779137
min,10004.000000,2022-01-02 00:00:00,2023-09-23 00:00:00,2023-01-02 00:00:00,2024-01-02 00:00:00,240.000000,240.000000,0.657534,0.657534
25%,12236.000000,2022-02-06 00:00:00,2024-07-28 00:00:00,2023-02-06 00:00:00,2024-02-06 00:00:00,240.000000,240.000000,0.657534,0.657534
50%,14925.000000,2022-04-06 00:00:00,2024-11-08 00:00:00,2023-04-06 00:00:00,2024-04-06 00:00:00,270.000000,270.000000,0.739726,0.739726
75%,17187.000000,2022-07-02 00:00:00,2025-01-12 00:00:00,2023-07-02 00:00:00,2024-07-02 00:00:00,330.000000,300.000000,0.904110,0.821918
max,19999.000000,2022-12-27 00:00:00,2025-03-28 00:00:00,2023-12-27 00:00:00,2024-12-27 00:00:00,630.000000,630.000000,1.726027,1.726027
std,2882.985403,NaN,NaN,NaN,NaN,68.520074,57.208398,0.187726,0.156735


In [19]:
med_agg = med_members_glp_22_24.pivot_table(index='MEMBER_ID', columns='PAID_YEAR', values='MEDICAL_PAID_AMOUNT', aggfunc='sum')
med_agg.reset_index(inplace=True)
med_agg.columns.name = None
med_agg.columns = [f"{col}" if col != 'PAID_YEAR' else 'PAID_YEAR' for col in med_agg.columns]
med_agg.rename(columns={'2021': 'Med_2021', '2022': 'Med_2022', '2023': 'Med_2023', '2024': 'Med_2024'}, inplace=True)
med_agg1 = med_agg
med_agg1

,MEMBER_ID,Med_2021,Med_2022,Med_2023,Med_2024
0,10000,767.16,74.21,418.39,8.22
1,10001,181.25,1207.51,737.43,924.46
2,10002,381.58,494.66,406.07,91.55
3,10003,NaN,231.70,NaN,411.31
4,10004,1022.38,872.68,1329.39,689.23
...,...,...,...,...,...
9995,19995,146.98,156.59,385.34,NaN
9996,19996,NaN,459.20,145.45,NaN
9997,19997,231.15,298.27,1056.58,1452.52
9998,19998,965.03,NaN,588.53,260.33


In [20]:
# Define inflation rates (as factors)
inflation_factors = {
    2021: 1.175, 
    2022: 1.117,   
    2023: 1.069, 
    2024: 1
}

# Normalize employer paid amounts for inflation
for year, factor in inflation_factors.items():
    column_name = f'Med_{year}'
    med_agg[column_name] = med_agg[column_name] * factor
med_agg

,MEMBER_ID,Med_2021,Med_2022,Med_2023,Med_2024
0,10000,901.41300,82.89257,447.25891,8.22
1,10001,212.96875,1348.78867,788.31267,924.46
2,10002,448.35650,552.53522,434.08883,91.55
3,10003,NaN,258.80890,NaN,411.31
4,10004,1201.29650,974.78356,1421.11791,689.23
...,...,...,...,...,...
9995,19995,172.70150,174.91103,411.92846,NaN
9996,19996,NaN,512.92640,155.48605,NaN
9997,19997,271.60125,333.16759,1129.48402,1452.52
9998,19998,1133.91025,NaN,629.13857,260.33


In [21]:
med_agg['Med_Delta'] = med_agg['Med_2024'] - med_agg['Med_2021']
med_agg['Med_%_Change'] =(( med_agg['Med_2024'] - med_agg['Med_2021'])/ med_agg['Med_2021']) * 100
med_agg

,MEMBER_ID,Med_2021,Med_2022,Med_2023,Med_2024,Med_Delta,Med_%_Change
0,10000,901.41300,82.89257,447.25891,8.22,-893.19300,-99.088098
1,10001,212.96875,1348.78867,788.31267,924.46,711.49125,334.082465
2,10002,448.35650,552.53522,434.08883,91.55,-356.80650,-79.580981
3,10003,NaN,258.80890,NaN,411.31,NaN,NaN
4,10004,1201.29650,974.78356,1421.11791,689.23,-512.06650,-42.626154
...,...,...,...,...,...,...,...
9995,19995,172.70150,174.91103,411.92846,NaN,NaN,NaN
9996,19996,NaN,512.92640,155.48605,NaN,NaN,NaN
9997,19997,271.60125,333.16759,1129.48402,1452.52,1180.91875,434.798717
9998,19998,1133.91025,NaN,629.13857,260.33,-873.58025,-77.041393


In [22]:
med_agg.fillna(0, inplace=True)

In [23]:
med_agg.isnull().sum()

MEMBER_ID       0
Med_2021        0
Med_2022        0
Med_2023        0
Med_2024        0
Med_Delta       0
Med_%_Change    0
dtype: int64

In [24]:
import numpy as np

# Prepare the years and medical costs
years = np.array([2021, 2022, 2023, 2024])
slope_list = []

# Calculate the slope for each member
for index, row in med_agg.iterrows():
    costs = np.array([row['Med_2021'], row['Med_2022'], row['Med_2023'], row['Med_2024']])
    # Perform linear regression to get the slope
    slope, intercept = np.polyfit(years, costs, 1)  # 1 indicates linear fit
    slope_list.append(slope)

# Add the slope to the DataFrame
med_agg['Med_Slope'] = slope_list
med_agg

,MEMBER_ID,Med_2021,Med_2022,Med_2023,Med_2024,Med_Delta,Med_%_Change,Med_Slope
0,10000,901.41300,82.89257,447.25891,8.22,-893.19300,-99.088098,-231.521266
1,10001,212.96875,1348.78867,788.31267,924.46,711.49125,334.082465,157.399775
2,10002,448.35650,552.53522,434.08883,91.55,-356.80650,-79.580981,-118.886589
3,10003,0.00000,258.80890,0.00000,411.31,0.00000,0.000000,97.512110
4,10004,1201.29650,974.78356,1421.11791,689.23,-512.06650,-42.626154,-108.986515
...,...,...,...,...,...,...,...,...
9995,19995,172.70150,174.91103,411.92846,0.00,0.00000,0.000000,-28.108707
9996,19996,0.00000,512.92640,155.48605,0.00,0.00000,0.000000,-35.744035
9997,19997,271.60125,333.16759,1129.48402,1452.52,1180.91875,434.798717,433.907268
9998,19998,1133.91025,0.00000,629.13857,260.33,-873.58025,-77.041393,-199.160218


In [25]:
rx_agg = rx_members_glp_22_24.pivot_table(index='MEMBER_ID', columns='PAID_YEAR', values='RX_PAID_AMOUNT', aggfunc='sum')
rx_agg.reset_index(inplace=True)
rx_agg.columns.name = None
rx_agg.columns = [f"{col}" if col != 'PAID_YEAR' else 'PAID_YEAR' for col in rx_agg.columns]
rx_agg.rename(columns={'2021': 'Rx_2021', '2022': 'Rx_2022', '2023': 'Rx_2023', '2024': 'Rx_2024'}, inplace=True)
rx_agg1 = rx_agg
rx_agg1

,MEMBER_ID,Rx_2021,Rx_2022,Rx_2023,Rx_2024
0,10000,171.57,17.50,656.53,582.07
1,10001,507.90,275.44,29.99,324.83
2,10002,257.12,630.90,576.03,37.71
3,10003,167.59,139.88,177.32,970.59
4,10004,1004.31,262.22,205.54,683.40
...,...,...,...,...,...
9995,19995,71.70,221.72,371.74,75.95
9996,19996,593.37,NaN,216.39,NaN
9997,19997,802.74,842.35,413.98,83.28
9998,19998,356.63,124.88,94.28,880.66


In [26]:
# Define inflation rates (as factors)
rx_inflation_factors = {
    2021: 1.31, 
    2022: 1.20,   
    2023: 1.103, 
    2024: 1
}

# Normalize employer paid amounts for inflation
for year, factor in rx_inflation_factors.items():
    column_name = f'Rx_{year}'
    rx_agg[column_name] = rx_agg[column_name] * factor
rx_agg

,MEMBER_ID,Rx_2021,Rx_2022,Rx_2023,Rx_2024
0,10000,224.7567,21.000,724.15259,582.07
1,10001,665.3490,330.528,33.07897,324.83
2,10002,336.8272,757.080,635.36109,37.71
3,10003,219.5429,167.856,195.58396,970.59
4,10004,1315.6461,314.664,226.71062,683.40
...,...,...,...,...,...
9995,19995,93.9270,266.064,410.02922,75.95
9996,19996,777.3147,NaN,238.67817,NaN
9997,19997,1051.5894,1010.820,456.61994,83.28
9998,19998,467.1853,149.856,103.99084,880.66


In [27]:
rx_agg.fillna(0, inplace=True)
rx_agg.isnull().sum()

MEMBER_ID    0
Rx_2021      0
Rx_2022      0
Rx_2023      0
Rx_2024      0
dtype: int64

In [28]:
pd.set_option('display.float_format', '{:.2f}'.format)

In [29]:
rx_agg['Rx_Delta'] = rx_agg['Rx_2024'] - rx_agg['Rx_2021']
rx_agg['Rx_%_Change'] =((rx_agg['Rx_2024'] - rx_agg['Rx_2021'])/ rx_agg['Rx_2021']) * 100
# Prepare the years and medical costs
years = np.array([2021, 2022, 2023, 2024])
slope_list = []

# Calculate the slope for each member
for index, row in rx_agg.iterrows():
    costs = np.array([row['Rx_2021'], row['Rx_2022'], row['Rx_2023'], row['Rx_2024']])
    # Perform linear regression to get the slope
    slope, intercept = np.polyfit(years, costs, 1)  # 1 indicates linear fit
    slope_list.append(slope)

# Add the slope to the DataFrame
rx_agg['Rx_Slope'] = slope_list
rx_agg

,MEMBER_ID,Rx_2021,Rx_2022,Rx_2023,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope
0,10000,224.76,21.00,724.15,582.07,357.31,158.98,177.51
1,10001,665.35,330.53,33.08,324.83,-340.52,-51.18,-131.90
2,10002,336.83,757.08,635.36,37.71,-299.12,-88.80,-101.91
3,10003,219.54,167.86,195.58,970.59,751.05,342.10,228.09
4,10004,1315.65,314.66,226.71,683.40,-632.25,-48.06,-198.47
...,...,...,...,...,...,...,...,...
9995,19995,93.93,266.06,410.03,75.95,-17.98,-19.14,9.00
9996,19996,777.31,0.00,238.68,0.00,-777.31,-100.00,-209.33
9997,19997,1051.59,1010.82,456.62,83.28,-968.31,-92.08,-345.91
9998,19998,467.19,149.86,103.99,880.66,413.47,88.50,119.46


In [30]:
rx_agg.replace(np.inf, 0, inplace=True)
rx_agg

,MEMBER_ID,Rx_2021,Rx_2022,Rx_2023,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope
0,10000,224.76,21.00,724.15,582.07,357.31,158.98,177.51
1,10001,665.35,330.53,33.08,324.83,-340.52,-51.18,-131.90
2,10002,336.83,757.08,635.36,37.71,-299.12,-88.80,-101.91
3,10003,219.54,167.86,195.58,970.59,751.05,342.10,228.09
4,10004,1315.65,314.66,226.71,683.40,-632.25,-48.06,-198.47
...,...,...,...,...,...,...,...,...
9995,19995,93.93,266.06,410.03,75.95,-17.98,-19.14,9.00
9996,19996,777.31,0.00,238.68,0.00,-777.31,-100.00,-209.33
9997,19997,1051.59,1010.82,456.62,83.28,-968.31,-92.08,-345.91
9998,19998,467.19,149.86,103.99,880.66,413.47,88.50,119.46


In [31]:
member_mrg = members_glp_22_24.merge(med_agg, on='MEMBER_ID')
member_mrg = member_mrg.merge(rx_agg, on='MEMBER_ID')                                     
member_mrg                                    

,MEMBER_ID,MEMBER_STATUS,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,Age,GENERATIONS,Med_2021,Med_2022,Med_2023,...,Med_Delta,Med_%_Change,Med_Slope,Rx_2021,Rx_2022,Rx_2023,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope
0,16252,2,CA,Subscriber,Male,56,Generation X,1005.98,837.98,403.64,...,-463.05,-46.03,-182.35,856.10,712.49,1120.21,151.37,-704.73,-82.32,-170.65
1,14684,3,NC,Domestic Partner,Male,73,Baby Boomers,1325.12,489.37,1008.46,...,-469.25,-35.41,-88.87,536.31,6.46,820.23,919.51,383.20,71.45,196.34
2,11731,2,WA,Spouse,Unknown,71,Baby Boomers,43.49,113.36,1067.75,...,5.46,12.56,97.08,698.10,121.57,217.68,167.41,-530.69,-76.02,-149.60
3,14742,3,FL,Subscriber,Female,55,Generation X,226.05,565.43,971.43,...,1156.00,511.40,387.40,614.34,0.00,970.97,289.90,-324.44,-52.81,-0.23
4,14521,1,FL,Subscriber,Unknown,39,Millenials,635.18,341.87,67.05,...,0.00,0.00,-218.04,1253.57,462.85,3.85,470.96,-782.61,-62.43,-280.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,15734,2,WA,Subscriber,Unknown,46,Generation X,17.95,763.77,110.36,...,249.74,1390.98,9.58,1178.53,58.45,5.34,704.52,-474.01,-40.22,-147.51
9996,15191,3,CA,Dependent,Male,15,Generation Z,75.98,488.50,103.93,...,476.38,627.02,104.46,184.19,61.50,1137.34,766.86,582.67,316.35,282.39
9997,15390,2,NY,Subscriber,Unknown,43,Millenials,982.12,153.24,105.23,...,-846.67,-86.21,-258.80,40.52,283.22,370.98,151.55,111.03,274.03,42.09
9998,10860,1,UT,Dependent,Unknown,42,Millenials,16.24,263.03,25.43,...,631.05,3886.14,165.56,244.88,468.73,70.92,177.34,-67.54,-27.58,-60.04


In [32]:
member1 = member_mrg.drop(columns=['MEMBER_STATUS'])
member1

,MEMBER_ID,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,Age,GENERATIONS,Med_2021,Med_2022,Med_2023,Med_2024,Med_Delta,Med_%_Change,Med_Slope,Rx_2021,Rx_2022,Rx_2023,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope
0,16252,CA,Subscriber,Male,56,Generation X,1005.98,837.98,403.64,542.93,-463.05,-46.03,-182.35,856.10,712.49,1120.21,151.37,-704.73,-82.32,-170.65
1,14684,NC,Domestic Partner,Male,73,Baby Boomers,1325.12,489.37,1008.46,855.87,-469.25,-35.41,-88.87,536.31,6.46,820.23,919.51,383.20,71.45,196.34
2,11731,WA,Spouse,Unknown,71,Baby Boomers,43.49,113.36,1067.75,48.95,5.46,12.56,97.08,698.10,121.57,217.68,167.41,-530.69,-76.02,-149.60
3,14742,FL,Subscriber,Female,55,Generation X,226.05,565.43,971.43,1382.05,1156.00,511.40,387.40,614.34,0.00,970.97,289.90,-324.44,-52.81,-0.23
4,14521,FL,Subscriber,Unknown,39,Millenials,635.18,341.87,67.05,0.00,0.00,0.00,-218.04,1253.57,462.85,3.85,470.96,-782.61,-62.43,-280.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,15734,WA,Subscriber,Unknown,46,Generation X,17.95,763.77,110.36,267.69,249.74,1390.98,9.58,1178.53,58.45,5.34,704.52,-474.01,-40.22,-147.51
9996,15191,CA,Dependent,Male,15,Generation Z,75.98,488.50,103.93,552.36,476.38,627.02,104.46,184.19,61.50,1137.34,766.86,582.67,316.35,282.39
9997,15390,NY,Subscriber,Unknown,43,Millenials,982.12,153.24,105.23,135.45,-846.67,-86.21,-258.80,40.52,283.22,370.98,151.55,111.03,274.03,42.09
9998,10860,UT,Dependent,Unknown,42,Millenials,16.24,263.03,25.43,647.29,631.05,3886.14,165.56,244.88,468.73,70.92,177.34,-67.54,-27.58,-60.04


In [33]:
member1.HOME_STATE.value_counts()

HOME_STATE
NY    1051
IL    1030
CA    1005
FL     998
WA     997
UT     995
PA     993
TX     988
CO     985
NC     958
Name: count, dtype: int64

In [34]:
# Define the states to keep
states_to_keep = ['NC', 'PA', 'TX', 'NJ', 'MS', 'OH', 'MI']
member1['HOME_STATE'] = member1['HOME_STATE'].apply(lambda x: x if x in states_to_keep else 'OTHER')
member1

,MEMBER_ID,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,Age,GENERATIONS,Med_2021,Med_2022,Med_2023,Med_2024,Med_Delta,Med_%_Change,Med_Slope,Rx_2021,Rx_2022,Rx_2023,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope
0,16252,OTHER,Subscriber,Male,56,Generation X,1005.98,837.98,403.64,542.93,-463.05,-46.03,-182.35,856.10,712.49,1120.21,151.37,-704.73,-82.32,-170.65
1,14684,NC,Domestic Partner,Male,73,Baby Boomers,1325.12,489.37,1008.46,855.87,-469.25,-35.41,-88.87,536.31,6.46,820.23,919.51,383.20,71.45,196.34
2,11731,OTHER,Spouse,Unknown,71,Baby Boomers,43.49,113.36,1067.75,48.95,5.46,12.56,97.08,698.10,121.57,217.68,167.41,-530.69,-76.02,-149.60
3,14742,OTHER,Subscriber,Female,55,Generation X,226.05,565.43,971.43,1382.05,1156.00,511.40,387.40,614.34,0.00,970.97,289.90,-324.44,-52.81,-0.23
4,14521,OTHER,Subscriber,Unknown,39,Millenials,635.18,341.87,67.05,0.00,0.00,0.00,-218.04,1253.57,462.85,3.85,470.96,-782.61,-62.43,-280.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,15734,OTHER,Subscriber,Unknown,46,Generation X,17.95,763.77,110.36,267.69,249.74,1390.98,9.58,1178.53,58.45,5.34,704.52,-474.01,-40.22,-147.51
9996,15191,OTHER,Dependent,Male,15,Generation Z,75.98,488.50,103.93,552.36,476.38,627.02,104.46,184.19,61.50,1137.34,766.86,582.67,316.35,282.39
9997,15390,OTHER,Subscriber,Unknown,43,Millenials,982.12,153.24,105.23,135.45,-846.67,-86.21,-258.80,40.52,283.22,370.98,151.55,111.03,274.03,42.09
9998,10860,OTHER,Dependent,Unknown,42,Millenials,16.24,263.03,25.43,647.29,631.05,3886.14,165.56,244.88,468.73,70.92,177.34,-67.54,-27.58,-60.04


In [35]:
med.columns

Index(['MEDICAL_CLAIM_ID', 'MEMBER_ID', 'PAID_DATE', 'PAID_YEAR',
       'SERVICE_DATE', 'SERVICE_YEAR', 'MEDICAL_PAID_AMOUNT',
       'IS_TELEMEDICINE', 'IS_ER_AVOIDABLE', 'PRIMARY_DX', 'PROCEDURE_DESC',
       'ARTTOS_V2_L1', 'ARTTOS_V2_L3', 'DX_IS_CHRONIC', 'ICD10_CHAPTER',
       'ICD10_CATEGORY', 'ICD10_SECTION', 'MEG_EPISODE_DESCRIPTION',
       'OP_SURG_INC', 'IS_PCP_VISIT', 'INCLUDED_SPECIALIST', 'ER_VISIT_FLAG',
       'IS_URGENT_CARE_VISIT', 'IS_PREVENTIVE_VISIT', 'IP_ADMIT_INC'],
      dtype='object')

In [36]:
med_util = med.groupby('MEMBER_ID').agg({
    'DX_IS_CHRONIC': 'max',
    'ER_VISIT_FLAG': 'sum',
    'IS_PCP_VISIT': 'sum',
    'IS_URGENT_CARE_VISIT': 'sum',
    'IS_PREVENTIVE_VISIT': 'sum',
    'IP_ADMIT_INC': 'sum'
}).reset_index()
med_util

,MEMBER_ID,DX_IS_CHRONIC,ER_VISIT_FLAG,IS_PCP_VISIT,IS_URGENT_CARE_VISIT,IS_PREVENTIVE_VISIT,IP_ADMIT_INC
0,10000,1,5,5,5,5,4
1,10001,1,7,7,10,5,6
2,10002,1,6,5,7,6,6
3,10003,1,3,5,5,5,3
4,10004,1,8,4,11,8,8
...,...,...,...,...,...,...,...
9995,19995,1,4,5,6,4,5
9996,19996,1,4,5,4,2,3
9997,19997,1,9,5,11,8,4
9998,19998,1,5,4,7,3,5


In [37]:
episodes = med.ICD10_CHAPTER.value_counts()
episodes.head(10)

ICD10_CHAPTER
Digestive      33490
Circulatory    33285
Endocrine      33225
Name: count, dtype: int64

In [38]:
episodes_cost = med.groupby('ICD10_CHAPTER')['MEDICAL_PAID_AMOUNT'].sum().reset_index()
episodes_cost.sort_values(by='MEDICAL_PAID_AMOUNT', ascending=False)

,ICD10_CHAPTER,MEDICAL_PAID_AMOUNT
2,Endocrine,6703577.36
1,Digestive,6692078.46
0,Circulatory,6655438.42


In [39]:
episodes_to_keep = ['Diseases of the musculoskeletal system and connective tissue', 'Neoplasms', 'Diseases of the digestive system', 
                    'Diseases of the circulatory system', 'Diseases of the circulatory system', 'Diseases of the genitourinary system',
                   'Diseases of the nervous system', 'Mental, Behavioral and Neurodevelopmental disorders']
med1 = med
med1['ICD10_CHAPTER'] = med['ICD10_CHAPTER'].apply(lambda x: x if x in episodes_to_keep else 'OTHER')

In [40]:
med_meg = pd.pivot_table(
    med1,
    values='MEDICAL_PAID_AMOUNT',
    index='MEMBER_ID',
    columns='ICD10_CHAPTER',
    aggfunc='sum',
    fill_value=0
)
med_meg.reset_index()
med_meg.columns.name = None
med_meg.drop(columns='OTHER', inplace=True)
med_meg

""
MEMBER_ID
10000
10001
10002
10003
10004
...
19995
19996
19997


In [41]:
med_var = member1.merge(med_util, on='MEMBER_ID')
med_var = med_var.merge(med_meg, on='MEMBER_ID')
med_var.columns= med_var.columns.str.replace(' ', '_')
med_var

,MEMBER_ID,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,Age,GENERATIONS,Med_2021,Med_2022,Med_2023,Med_2024,...,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope,DX_IS_CHRONIC,ER_VISIT_FLAG,IS_PCP_VISIT,IS_URGENT_CARE_VISIT,IS_PREVENTIVE_VISIT,IP_ADMIT_INC
0,16252,OTHER,Subscriber,Male,56,Generation X,1005.98,837.98,403.64,542.93,...,151.37,-704.73,-82.32,-170.65,1,5,5,7,8,6
1,14684,NC,Domestic Partner,Male,73,Baby Boomers,1325.12,489.37,1008.46,855.87,...,919.51,383.20,71.45,196.34,1,6,8,9,6,6
2,11731,OTHER,Spouse,Unknown,71,Baby Boomers,43.49,113.36,1067.75,48.95,...,167.41,-530.69,-76.02,-149.60,1,4,3,4,2,4
3,14742,OTHER,Subscriber,Female,55,Generation X,226.05,565.43,971.43,1382.05,...,289.90,-324.44,-52.81,-0.23,1,7,7,6,6,7
4,14521,OTHER,Subscriber,Unknown,39,Millenials,635.18,341.87,67.05,0.00,...,470.96,-782.61,-62.43,-280.68,1,4,3,5,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,15734,OTHER,Subscriber,Unknown,46,Generation X,17.95,763.77,110.36,267.69,...,704.52,-474.01,-40.22,-147.51,1,8,3,2,5,3
9996,15191,OTHER,Dependent,Male,15,Generation Z,75.98,488.50,103.93,552.36,...,766.86,582.67,316.35,282.39,1,6,4,6,4,7
9997,15390,OTHER,Subscriber,Unknown,43,Millenials,982.12,153.24,105.23,135.45,...,151.55,111.03,274.03,42.09,1,5,2,4,2,2
9998,10860,OTHER,Dependent,Unknown,42,Millenials,16.24,263.03,25.43,647.29,...,177.34,-67.54,-27.58,-60.04,1,4,4,3,5,4


In [42]:
mbr = members_65['MEMBER_ID'].unique()
members65_df = med_var[med_var['MEMBER_ID'].isin(mbr)]
members65_df

,MEMBER_ID,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,Age,GENERATIONS,Med_2021,Med_2022,Med_2023,Med_2024,...,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope,DX_IS_CHRONIC,ER_VISIT_FLAG,IS_PCP_VISIT,IS_URGENT_CARE_VISIT,IS_PREVENTIVE_VISIT,IP_ADMIT_INC
26,11963,OTHER,Subscriber,Female,45,Generation X,28.85,1064.56,1197.59,464.60,...,0.00,-330.08,-100.00,-47.70,1,6,4,4,3,8
27,18243,OTHER,Subscriber,Unknown,27,Generation Z,1416.37,990.81,933.74,612.60,...,377.80,-633.70,-62.65,-237.08,1,11,10,7,11,13
71,10799,NC,Dependent,Female,41,Millenials,892.10,574.75,1106.90,185.24,...,452.59,-838.66,-64.95,-207.92,1,8,9,12,7,13
101,15019,OTHER,Subscriber,Female,30,Millenials,298.80,868.53,501.52,340.70,...,699.66,253.42,56.79,35.46,1,3,7,7,7,7
193,12908,OTHER,Unknown,Male,59,Generation X,610.51,1080.15,145.13,359.04,...,946.00,860.08,1000.99,225.76,1,9,6,9,8,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9847,11306,NC,Dependent,Male,45,Generation X,91.47,445.54,54.68,1073.18,...,236.34,-241.25,-50.51,-70.27,1,4,3,5,2,4
9862,13242,OTHER,Dependent,Unknown,47,Generation X,2124.73,718.48,225.80,431.00,...,373.21,-1127.51,-75.13,-378.52,1,7,6,5,8,4
9874,17858,OTHER,Unknown,Female,28,Millenials,199.87,86.80,255.33,1242.55,...,312.43,165.00,111.92,69.44,1,6,6,4,4,3
9885,10502,OTHER,Dependent,Male,62,Baby Boomers,164.14,93.19,2013.14,730.81,...,444.10,-1075.88,-70.78,-341.11,1,6,7,9,3,8


In [43]:
med_var.columns

Index(['MEMBER_ID', 'HOME_STATE', 'MEMBER_RELATIONSHIP', 'GENDER', 'Age',
       'GENERATIONS', 'Med_2021', 'Med_2022', 'Med_2023', 'Med_2024',
       'Med_Delta', 'Med_%_Change', 'Med_Slope', 'Rx_2021', 'Rx_2022',
       'Rx_2023', 'Rx_2024', 'Rx_Delta', 'Rx_%_Change', 'Rx_Slope',
       'DX_IS_CHRONIC', 'ER_VISIT_FLAG', 'IS_PCP_VISIT',
       'IS_URGENT_CARE_VISIT', 'IS_PREVENTIVE_VISIT', 'IP_ADMIT_INC'],
      dtype='object')

In [44]:
members65_df.describe()

,MEMBER_ID,Age,Med_2021,Med_2022,Med_2023,Med_2024,Med_Delta,Med_%_Change,Med_Slope,Rx_2021,...,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope,DX_IS_CHRONIC,ER_VISIT_FLAG,IS_PCP_VISIT,IS_URGENT_CARE_VISIT,IS_PREVENTIVE_VISIT,IP_ADMIT_INC
count,317.00,317.00,317.00,317.00,317.00,317.00,317.00,317.00,317.00,317.00,...,317.00,317.00,317.00,317.00,317.00,317.00,317.00,317.00,317.00,317.00
mean,14788.67,44.64,895.03,766.87,777.16,745.83,-142.05,165.39,-43.73,726.51,...,518.62,-207.89,143.39,-63.77,1.00,7.21,7.37,7.19,7.17,7.45
std,2882.99,19.00,689.18,551.35,555.59,518.93,873.49,978.48,276.35,535.65,...,344.60,658.49,886.15,207.85,0.00,2.26,2.39,2.34,2.50,2.47
min,10004.00,13.00,0.00,0.00,0.00,0.00,-3930.68,-99.41,-1117.71,0.00,...,0.00,-3408.62,-100.00,-1081.87,1.00,3.00,2.00,3.00,0.00,1.00
25%,12236.00,30.00,395.48,391.24,348.39,353.17,-670.96,-59.00,-209.05,347.77,...,252.08,-579.85,-65.89,-185.01,1.00,6.00,5.00,5.00,5.00,6.00
50%,14925.00,42.00,743.15,653.39,650.96,656.74,-58.25,-10.02,-24.13,627.74,...,444.10,-147.65,-26.72,-44.76,1.00,7.00,8.00,7.00,7.00,7.00
75%,17187.00,61.00,1228.06,1025.81,1061.39,1043.24,421.09,104.05,144.03,991.67,...,735.71,225.04,48.93,64.64,1.00,8.00,9.00,9.00,9.00,9.00
max,19999.00,79.00,4281.97,2605.56,3229.15,2899.88,2031.63,14182.68,633.90,3534.92,...,1614.06,1240.34,11891.43,446.83,1.00,15.00,14.00,15.00,14.00,15.00


In [45]:
costs = members65_df[['MEMBER_ID', 'HOME_STATE', 'MEMBER_RELATIONSHIP', 'GENDER', 'Age',
       'GENERATIONS', 'Med_2021', 'Med_2022', 'Med_2023', 'Med_2024',
       'Med_Delta', 'Med_%_Change', 'Med_Slope', 'Rx_2021', 'Rx_2022',
       'Rx_2023', 'Rx_2024', 'Rx_Delta', 'Rx_%_Change', 'Rx_Slope']]
costs

,MEMBER_ID,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,Age,GENERATIONS,Med_2021,Med_2022,Med_2023,Med_2024,Med_Delta,Med_%_Change,Med_Slope,Rx_2021,Rx_2022,Rx_2023,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope
26,11963,OTHER,Subscriber,Female,45,Generation X,28.85,1064.56,1197.59,464.60,435.75,1510.61,144.03,330.08,269.88,783.14,0.00,-330.08,-100.00,-47.70
27,18243,OTHER,Subscriber,Unknown,27,Generation Z,1416.37,990.81,933.74,612.60,-803.77,-56.75,-246.84,1011.50,1220.08,750.40,377.80,-633.70,-62.65,-237.08
71,10799,NC,Dependent,Female,41,Millenials,892.10,574.75,1106.90,185.24,-706.86,-79.24,-158.84,1291.25,701.95,1138.77,452.59,-838.66,-64.95,-207.92
101,15019,OTHER,Subscriber,Female,30,Millenials,298.80,868.53,501.52,340.70,41.90,14.02,-24.13,446.24,510.94,105.28,699.66,253.42,56.79,35.46
193,12908,OTHER,Unknown,Male,59,Generation X,610.51,1080.15,145.13,359.04,-251.47,-41.19,-168.94,85.92,694.15,371.53,946.00,860.08,1000.99,225.76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9847,11306,NC,Dependent,Male,45,Generation X,91.47,445.54,54.68,1073.18,981.71,1073.21,255.43,477.59,247.18,268.23,236.34,-241.25,-50.51,-70.27
9862,13242,OTHER,Dependent,Unknown,47,Generation X,2124.73,718.48,225.80,431.00,-1693.73,-79.72,-557.39,1500.72,553.98,151.32,373.21,-1127.51,-75.13,-378.52
9874,17858,OTHER,Unknown,Female,28,Millenials,199.87,86.80,255.33,1242.55,1042.68,521.69,329.66,147.43,430.44,629.88,312.43,165.00,111.92,69.44
9885,10502,OTHER,Dependent,Male,62,Baby Boomers,164.14,93.19,2013.14,730.81,566.67,345.25,362.00,1519.98,595.08,411.57,444.10,-1075.88,-70.78,-341.11


In [46]:
costs.describe()

,MEMBER_ID,Age,Med_2021,Med_2022,Med_2023,Med_2024,Med_Delta,Med_%_Change,Med_Slope,Rx_2021,Rx_2022,Rx_2023,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope
count,317.00,317.00,317.00,317.00,317.00,317.00,317.00,317.00,317.00,317.00,317.00,317.00,317.00,317.00,317.00,317.00
mean,14788.67,44.64,895.03,766.87,777.16,745.83,-142.05,165.39,-43.73,726.51,601.78,587.76,518.62,-207.89,143.39,-63.77
std,2882.99,19.00,689.18,551.35,555.59,518.93,873.49,978.48,276.35,535.65,459.26,415.78,344.60,658.49,886.15,207.85
min,10004.00,13.00,0.00,0.00,0.00,0.00,-3930.68,-99.41,-1117.71,0.00,0.00,0.00,0.00,-3408.62,-100.00,-1081.87
25%,12236.00,30.00,395.48,391.24,348.39,353.17,-670.96,-59.00,-209.05,347.77,252.79,268.23,252.08,-579.85,-65.89,-185.01
50%,14925.00,42.00,743.15,653.39,650.96,656.74,-58.25,-10.02,-24.13,627.74,528.88,527.22,444.10,-147.65,-26.72,-44.76
75%,17187.00,61.00,1228.06,1025.81,1061.39,1043.24,421.09,104.05,144.03,991.67,837.19,836.44,735.71,225.04,48.93,64.64
max,19999.00,79.00,4281.97,2605.56,3229.15,2899.88,2031.63,14182.68,633.90,3534.92,2738.60,2269.85,1614.06,1240.34,11891.43,446.83


average med+rx net per member, normalized for 2024 = $11,834-1,554 = $10,280. Since 2021, employers have spent $10,280 average per member on those that have prescriptions for Wegovy, Saxenda or Zepbound for 3 years

In [47]:
members65_df.to_csv('glp1_members_pdc65.csv', index=False)

In [52]:
drugs = rx.groupby(['ART_DRUGGRPS_L1', 'ART_DRUGGRPS_L2', 'DRUG_NAME_PREFERRED'])['RX_PAID_AMOUNT'].sum().reset_index()
drugs.sort_values(by='RX_PAID_AMOUNT', ascending=False, inplace=True)
drugs

,ART_DRUGGRPS_L1,ART_DRUGGRPS_L2,DRUG_NAME_PREFERRED,RX_PAID_AMOUNT
99,Obesity Agents,Beta Blockers,Zepbound,202429.04
145,Respiratory Agents,GLP-1 Agonists,Saxenda,201586.64
138,Respiratory Agents,Beta Blockers,Wegovy,199659.79
95,Obesity Agents,Beta Blockers,Saxenda,197164.16
149,Respiratory Agents,GLP-1 Agonists,Zepbound,197089.88
...,...,...,...,...
91,Obesity Agents,Beta Blockers,Lipitor,46789.39
71,Antihypertensives,Inhalers,Lipitor,46704.27
127,Respiratory Agents,ACE Inhibitors,Ventolin,46424.46
10,Antidiabetics,Beta Blockers,Glucophage,46197.28


In [49]:
med.to_csv('GLP-1_Medical_PDC.csv', index=False)